In [1]:
# -*- coding: utf-8 -*-
'''VGG16 model for Keras.
# Reference:
- [Very Deep Convolutional Networks for Large-Scale Image Recognition](https://arxiv.org/abs/1409.1556)
'''
from __future__ import print_function

import numpy as np
import warnings

from keras.models import Model
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Input
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import GlobalMaxPooling2D
from keras.layers import GlobalAveragePooling2D
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras import backend as K
from keras.applications.imagenet_utils import decode_predictions
from keras.applications.imagenet_utils import preprocess_input
from keras.applications.imagenet_utils import _obtain_input_shape
from keras.engine.topology import get_source_inputs


WEIGHTS_PATH = 'https://github.com/fchollet/deep-learning-models/releases/download/v0.1/vgg16_weights_tf_dim_ordering_tf_kernels.h5'
WEIGHTS_PATH_NO_TOP = 'https://github.com/fchollet/deep-learning-models/releases/download/v0.1/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'


def VGG16(include_top=True, weights='imagenet',
          input_tensor=None, input_shape=None,
          pooling=None,
          classes=1000):
    """Instantiates the VGG16 architecture.
    Optionally loads weights pre-trained
    on ImageNet. Note that when using TensorFlow,
    for best performance you should set
    `image_data_format="channels_last"` in your Keras config
    at ~/.keras/keras.json.
    The model and the weights are compatible with both
    TensorFlow and Theano. The data format
    convention used by the model is the one
    specified in your Keras config file.
    # Arguments
        include_top: whether to include the 3 fully-connected
            layers at the top of the network.
        weights: one of `None` (random initialization)
            or "imagenet" (pre-training on ImageNet).
        input_tensor: optional Keras tensor (i.e. output of `layers.Input()`)
            to use as image input for the model.
        input_shape: optional shape tuple, only to be specified
            if `include_top` is False (otherwise the input shape
            has to be `(224, 224, 3)` (with `channels_last` data format)
            or `(3, 224, 244)` (with `channels_first` data format).
            It should have exactly 3 inputs channels,
            and width and height should be no smaller than 48.
            E.g. `(200, 200, 3)` would be one valid value.
        pooling: Optional pooling mode for feature extraction
            when `include_top` is `False`.
            - `None` means that the output of the model will be
                the 4D tensor output of the
                last convolutional layer.
            - `avg` means that global average pooling
                will be applied to the output of the
                last convolutional layer, and thus
                the output of the model will be a 2D tensor.
            - `max` means that global max pooling will
                be applied.
        classes: optional number of classes to classify images
            into, only to be specified if `include_top` is True, and
            if no `weights` argument is specified.
    # Returns
        A Keras model instance.
    # Raises
        ValueError: in case of invalid argument for `weights`,
            or invalid input shape.
    """
    if weights not in {'imagenet', None}:
        raise ValueError('The `weights` argument should be either '
                         '`None` (random initialization) or `imagenet` '
                         '(pre-training on ImageNet).')

    if weights == 'imagenet' and include_top and classes != 1000:
        raise ValueError('If using `weights` as imagenet with `include_top`'
                         ' as true, `classes` should be 1000')
    # Determine proper input shape
    input_shape = _obtain_input_shape(input_shape,
                                      default_size=224,
                                      min_size=48,
                                      data_format=K.image_data_format(),
                                      include_top=include_top)

    if input_tensor is None:
        img_input = Input(shape=input_shape)
    else:
        if not K.is_keras_tensor(input_tensor):
            img_input = Input(tensor=input_tensor, shape=input_shape)
        else:
            img_input = input_tensor
    # Block 1
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1')(img_input)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

    # Block 2
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

    # Block 3
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

    # Block 4
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

    # Block 5
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv1')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv2')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x)

    if include_top:
        # Classification block
        x = Flatten(name='flatten')(x)
        x = Dense(4096, activation='relu', name='fc1')(x)
        x = Dense(4096, activation='relu', name='fc2')(x)
        x = Dense(classes, activation='softmax', name='predictions')(x)
    else:
        if pooling == 'avg':
            x = GlobalAveragePooling2D()(x)
        elif pooling == 'max':
            x = GlobalMaxPooling2D()(x)

    # Ensure that the model takes into account
    # any potential predecessors of `input_tensor`.
    if input_tensor is not None:
        inputs = get_source_inputs(input_tensor)
    else:
        inputs = img_input
    # Create model.
    model = Model(inputs, x, name='vgg16')

    # load weights
    if weights == 'imagenet':
        if include_top:
            weights_path = get_file('vgg16_weights_tf_dim_ordering_tf_kernels.h5',
                                    WEIGHTS_PATH,
                                    cache_subdir='models')
        else:
            weights_path = get_file('vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5',
                                    WEIGHTS_PATH_NO_TOP,
                                    cache_subdir='models')
        model.load_weights(weights_path)
        if K.backend() == 'theano':
            layer_utils.convert_all_kernels_in_model(model)

        if K.image_data_format() == 'channels_first':
            if include_top:
                maxpool = model.get_layer(name='block5_pool')
                shape = maxpool.output_shape[1:]
                dense = model.get_layer(name='fc1')
                layer_utils.convert_dense_weights_data_format(dense, shape, 'channels_first')

            if K.backend() == 'tensorflow':
                warnings.warn('You are using the TensorFlow backend, yet you '
                              'are using the Theano '
                              'image data format convention '
                              '(`image_data_format="channels_first"`). '
                              'For best performance, set '
                              '`image_data_format="channels_last"` in '
                              'your Keras config '
                              'at ~/.keras/keras.json.')
    return model




Using TensorFlow backend.
/Users/wenqi/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.7
  return f(*args, **kwds)


ImportError: cannot import name '_obtain_input_shape' from 'keras.applications.imagenet_utils' (/Users/wenqi/anaconda3/lib/python3.7/site-packages/keras/applications/imagenet_utils.py)

In [2]:
if __name__ == '__main__':
    model = VGG16(include_top=True, weights='imagenet')

    img_path = 'persian_cat.jpg'
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    print('Input image shape:', x.shape)

    preds = model.predict(x)
    print('Predicted:', decode_predictions(preds))
    print(np.argmax(preds))

NameError: name 'VGG16' is not defined

In [3]:
model.summary()

NameError: name 'model' is not defined

In [42]:
layer_names = ['block1_conv1',
         'block1_conv2',
         'block2_conv1',
         'block2_conv2',
         'block3_conv1',
         'block3_conv2',
         'block3_conv3',
         'block4_conv1',
         'block4_conv2',
         'block4_conv3',
         'block5_conv1',
         'block5_conv2',
         'block5_conv3',
         'fc1',
         'fc2',
         'predictions']

def my_reshape(x):
    """
    x: a conv layer of the pretrained model, 
        shape (filter_num, channel, row, col)
    
    output: reshaped conv weights
        shape (filter_num, row, col, channel)
    """
    row, col, channel, filter_num = x.shape
    output = np.zeros((filter_num, row, col, channel))
    
    for fil in range(filter_num):
        for ch in range(channel):
            for r in range(row):
                for c in range(col):
                    output[fil, r, c, ch] = x[r, c, ch, fil]
                    
    return output
    
layers = [model.get_layer(layer_name) for layer_name in layer_names]
np.array([[[[[1]]]]])
dir_name = './weights_npy/'
for i, layer in enumerate(layers):
    weight = layer.get_weights()[0]
    bias = layer.get_weights()[1]
    if len(weight.shape) == 4:
        weight = my_reshape(weight)
    print (weight.shape)
    np.save(dir_name + layer_names[i] + 'kernel', weight)
    np.save(dir_name + layer_names[i] + 'bias', bias)

(64, 3, 3, 3)
(64, 3, 3, 64)
(128, 3, 3, 64)
(128, 3, 3, 128)
(256, 3, 3, 128)
(256, 3, 3, 256)
(256, 3, 3, 256)
(512, 3, 3, 256)
(512, 3, 3, 512)
(512, 3, 3, 512)
(512, 3, 3, 512)
(512, 3, 3, 512)
(512, 3, 3, 512)
(25088, 4096)
(4096, 4096)
(4096, 1000)


In [36]:
%matplotlib inline
#The line above is necesary to show Matplotlib's plots inside a Jupyter Notebook

import cv2
from matplotlib import pyplot as plt
import matplotlib.image as mpimg

im = cv2.resize(cv2.imread('persian_cat.jpg'), (224, 224)).astype(np.float32)

im[:,:,0] -= 103.939
im[:,:,1] -= 116.779
im[:,:,2] -= 123.68
im = np.expand_dims(im, axis=0)
print(im.shape)
print(im)

(1, 224, 224, 3)
[[[[-102.93900299 -115.77899933 -122.68000031]
   [-102.93900299 -115.77899933 -122.68000031]
   [-103.93900299 -116.77899933 -123.68000031]
   ..., 
   [-101.93900299 -114.77899933 -121.68000031]
   [-101.93900299 -114.77899933 -121.68000031]
   [-102.93900299 -115.77899933 -122.68000031]]

  [[-102.93900299 -115.77899933 -122.68000031]
   [-102.93900299 -115.77899933 -122.68000031]
   [-103.93900299 -116.77899933 -123.68000031]
   ..., 
   [-101.93900299 -114.77899933 -121.68000031]
   [-101.93900299 -114.77899933 -121.68000031]
   [-102.93900299 -115.77899933 -122.68000031]]

  [[-102.93900299 -115.77899933 -122.68000031]
   [-102.93900299 -115.77899933 -122.68000031]
   [-103.93900299 -116.77899933 -123.68000031]
   ..., 
   [-101.93900299 -114.77899933 -121.68000031]
   [-101.93900299 -114.77899933 -121.68000031]
   [-102.93900299 -115.77899933 -122.68000031]]

  ..., 
  [[-103.93900299 -116.77899933 -123.68000031]
   [-102.93900299 -115.77899933 -122.68000031]
  

In [37]:
from keras.models import Model

layer_name = 'convolution2d_1'
intermediate_layer_model = Model(inputs=model.input, outputs=model.get_layer('block1_conv1').output)
intermediate_output = intermediate_layer_model.predict(im)

print(intermediate_output)
print(np.argmax(intermediate_output))

[[[[  3.69494507e+02   0.00000000e+00   9.88306999e+00 ...,
      1.35643784e+02   0.00000000e+00   2.18990307e+01]
   [  2.84514404e+02   0.00000000e+00   0.00000000e+00 ...,
      2.13458511e+02   0.00000000e+00   0.00000000e+00]
   [  2.85263031e+02   0.00000000e+00   0.00000000e+00 ...,
      2.13843796e+02   0.00000000e+00   0.00000000e+00]
   ..., 
   [  2.78810242e+02   0.00000000e+00   0.00000000e+00 ...,
      2.11466507e+02   0.00000000e+00   0.00000000e+00]
   [  2.82052002e+02   0.00000000e+00   0.00000000e+00 ...,
      2.11591171e+02   0.00000000e+00   0.00000000e+00]
   [  0.00000000e+00   0.00000000e+00   0.00000000e+00 ...,
      1.20852440e+02   0.00000000e+00   0.00000000e+00]]

  [[  2.53709534e+02   0.00000000e+00   4.49931183e+01 ...,
      2.18578522e+02   2.21781036e+02   1.56441559e+02]
   [  2.11737776e+00   2.37710452e+00   0.00000000e+00 ...,
      3.35787384e+02   2.51883054e+00   0.00000000e+00]
   [  1.64391291e+00   2.18792415e+00   0.00000000e+00 ...,
 

In [30]:
from keras.applications.imagenet_utils import decode_predictions

temp = np.zeros((1,1000))
temp[0][283] = 1
decode_predictions(temp)

[[(u'n02123394', u'Persian_cat', 1.0),
  (u'n15075141', u'toilet_tissue', 0.0),
  (u'n02319095', u'sea_urchin', 0.0),
  (u'n02391049', u'zebra', 0.0),
  (u'n02389026', u'sorrel', 0.0)]]